In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras import Sequential
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Concatenate
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError as RMSE

# Generación de datos ficticios
num_muestras = 1000  # Número de muestras en el conjunto de datos

# Características visuales: 128 características numéricas (x1)
caracteristicas_visuales = np.random.rand(num_muestras, 128)

# Características de texto: 256 características numéricas (x2)
caracteristicas_texto = np.random.rand(num_muestras, 256)

# Precio (Salida 1): Valor numérico (por ejemplo, precio de un producto) (y1)
precio = np.random.rand(num_muestras, 1)

# Categoría (Salida 2): 10 categorías posibles (codificadas en one-hot) y2
categorias = np.random.randint(0, 10, size=(num_muestras, 1))
categorias_one_hot = tf.keras.utils.to_categorical(categorias, num_classes=10)

x_temp_vis, x_test_vis, x_temp_txt, x_test_txt, y_temp_prec, y_test_prec, y_temp_cat, y_test_cat = train_test_split(
    caracteristicas_visuales,
    caracteristicas_texto,
    precio,
    categorias_one_hot,
    test_size=0.2,
    random_state=17)


x_train_vis, x_val_vis, x_train_txt, x_val_txt, y_train_prec, y_val_prec, y_train_cat, y_val_cat = train_test_split(
    x_temp_vis,
    x_temp_txt,
    y_temp_prec,
    y_temp_cat,
    test_size=0.2,
    random_state=17)

tf.keras.backend.clear_session()
tf.random.set_seed(17)

input_layer_visual = tf.keras.layers.Input(shape=[128],name="Input_Visual") 
input_layer_texto = tf.keras.layers.Input(shape=[256],name="Input_Texto")  
hidden_layer_visual = tf.keras.layers.Dense(64,name="Oculta_Visual",activation="relu")
hidden_layer_texto = tf.keras.layers.Dense(128,name="Oculta_Texto",activation="relu")
output_layer_categorico = tf.keras.layers.Dense(10,activation="softmax",name="Saldia_Categorica")
output_layer_precio = tf.keras.layers.Dense(1,name="Salida_Precio")

hidden_visual = hidden_layer_visual(input_layer_visual)
hidden_texto = hidden_layer_texto(input_layer_texto)
concat = tf.keras.layers.Concatenate()([hidden_visual, hidden_texto])
output_categorico = output_layer_categorico(hidden_texto)
output_precio = output_layer_precio(concat)

model = tf.keras.Model(inputs=[input_layer_visual, input_layer_texto], outputs=[output_categorico, output_precio])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
    loss={
        "salida_categoria": "categorical_crossentropy", # Clasificacion
        "salida_precio": "mse" # Regresion
    },
    metrics={
        "salida_categoria": ["accuracy"], # Clasificacion
        "salida_precio": ["mae"] # Regresion
    }
)
history=model.fit(
    [x_train_vis, x_train_txt],
    [y_train_cat, y_train_prec],
    epochs=30,
    batch_size=32,
    validation_data=((x_val_vis, x_val_txt), (y_val_cat, y_val_prec))
)


I0000 00:00:1738078363.487098   32655 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738078363.515124   32655 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738078363.515312   32655 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738078363.516935   32655 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/30


/home/iabd/anaconda3/envs/IABD3_9/lib/python3.9/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['Input_Visual', 'Input_Texto']. Received: the structure of inputs=('*', '*')
  warnings.warn(


KeyError: "There are keys: ['salida_categoria', 'salida_precio'] in the `loss` argument, but they can't be found in the model's output (`y_pred`)."